# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 1-1
## Учимся переводить между decision/evaluation/search на примере Bin Packing

**Даны:** 
* $[w_1, \dots , w_k], w_i \in [0, 1]$ — веса грузов  
* $n_{\text{bins}}$ — количество корзин грузоподъемностью 1.

Предполагается, что функция `solve_bp_decision(weights, n_bins)` решает «распознавательный» (decision) вариант задачи bin packing. На вход ей подаётся список весов и число корзин. Функция возвращает булевский ответ на вопрос «можно ли заданные веса раскидать по не более чем `n_bins` контейнерам? 

Напишите содержимое функции `solve_bp_evaluation`, которая возвращает оптимальное число корзин (решает evaluation variant задачи bin packing). 

Затем напишите содержимое функции `solve_bp_search`, которая возвращает список номеров корзин, которые при каком-нибудь оптимальном распределении присваиваются весам из списка `weights` (корзины нумеруются с единицы). 

Каждая из следующих функций должна вызывать предыдущую не более чем полиномиальное число раз.

In [50]:
def solve_bp_decision(weights: list, n_bins: int) -> bool:
    def able_to_pack(weights: list, bin_capacities) -> bool:
        return weights == [] or any( 
            able_to_pack( weights[:-1], bin_capacities[:i] + [capacity - weights[-1]] + bin_capacities[(i + 1):] ) 
            for i, capacity in enumerate(bin_capacities) if capacity >= weights[-1] 
        )

    return able_to_pack( sorted(weights), [1.0] * n_bins )

# To test this, run:
solve_bp_decision([0.8, 0.09, 0.4, 0.7], 2), solve_bp_decision([0.8, 0.09, 0.4, 0.7], 3)

(False, True)

In [58]:
def solve_bp_evaluation(weights: list) -> int:
    # Бинпоиск по ответу
    lower = 0
    upper = len(weights)
    while (upper - lower) > 1:   
        x = lower + (upper - lower) // 2
        if solve_bp_decision(weights, x):
            upper = x
        else:
            lower = x
    return upper        
#   … your code here
#   return min_n_bins
    pass

solve_bp_evaluation([0.8, 0.09, 0.4, 0.7]), solve_bp_evaluation([])

(3, 0)

In [56]:
def solve_bp_search(weights: list):
    solution = [-1] * len(weights)
    # Основная идея: если solve_bp_evaluation([x1, x2, x3, ...]) = solve_bp_evaluation([x1+x2, x3...]), 
    # то существует оптимальная укладка, в которой первый и второй грузы лежат в одной корзине. Тогда эти грузы можно 
    # склеить в один с весом x1+x2
    bins = [0] # корзины 
    for i in range(len(weights)):
        for j in range(len(bins)):
            if bins[j] + weights[i] > 1 and j == len(bins) - 1:
                bins.append(weights[i])
                solution[i] = len(bins)
            elif bins[j] + weights[i] > 1 and j < len(bins) - 1:
                continue
            else: 
                bins_1 = bins.copy()
                bins_1.append(weights[i])
                bins_2 = bins.copy()
                bins_2[j] += weights[i]
                if solve_bp_evaluation(bins_1) == solve_bp_evaluation(bins_2):
                    bins[j] += weights[i]
                    solution[i] = j + 1
                    break 
    return solution        
            
        

#   … your code here
#   return solution # list
    pass
solve_bp_search([0.72, 0.17, 0.09, 0.52, 0.31, 0.46, 0.89, 0.98]), solve_bp_search([0.8, 0.09, 0.4, 0.7]) 

([1, 1, 1, 2, 2, 3, 4, 5], [1, 1, 2, 3])